In [109]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.options import Options
import os
import json
import re
import datetime
import sys
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import requests, zipfile
from io import StringIO
from io import BytesIO

try: 
    from pyvirtualdisplay import Display
except  ModuleNotFoundError:
    !pip install PyVirtualDisplay
    
try: 
    from seleniumwire import webdriver  # Import from seleniumwire
except  ModuleNotFoundError:
    print("module selenium-wire is not installed, now start installation.")
    !pip install selenium-wire

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
url = 'https://data.binance.vision/?prefix=data/futures/um/daily/klines/'
driver.get(url)

In [5]:
_list = driver.find_element_by_id('listing')
rows = _list.find_elements(By.TAG_NAME, 'tr')
names = [row.find_elements(By.TAG_NAME, 'td')[0].text for row in rows]

In [6]:
futures_names = [name for name in names if name.find('_')!=-1]
print(len(futures_names))

16


In [5]:


# zipfile.ZipFile(StringIO(r.content))
futures_data_url_arr = []

for futures_name in futures_names:
    futures_data_url_arr.append(url+futures_name+'1h/')


NameError: name 'futures_names' is not defined

In [8]:
zip_data_name_arr = []

i=0
length = 0
for data_url in futures_data_url_arr:
    print(data_url)
    driver.get(data_url)
    driver.implicitly_wait(5)
    while length == 0 :
        _list = driver.find_element_by_id('listing')
        rows = _list.find_elements(By.TAG_NAME, 'tr')
        names = [row.find_elements(By.TAG_NAME, 'td')[0].text for row in rows]    
        names = [name for name in names if name.endswith('.zip')]
        zip_data_name_arr.append(names)
        length = len(names)
    length = 0
    i+=1
    print(i, len(futures_data_url_arr), len(names))

https://data.binance.vision/?prefix=data/futures/um/daily/klines/BTCBUSD_210129/1h/
1 16 8
https://data.binance.vision/?prefix=data/futures/um/daily/klines/BTCBUSD_210226/1h/
2 16 8
https://data.binance.vision/?prefix=data/futures/um/daily/klines/BTCUSDT_210326/1h/
3 16 52
https://data.binance.vision/?prefix=data/futures/um/daily/klines/BTCUSDT_210625/1h/
4 16 102
https://data.binance.vision/?prefix=data/futures/um/daily/klines/BTCUSDT_210924/1h/
5 16 98
https://data.binance.vision/?prefix=data/futures/um/daily/klines/BTCUSDT_211231/1h/
6 16 101
https://data.binance.vision/?prefix=data/futures/um/daily/klines/BTCUSDT_220325/1h/
7 16 92
https://data.binance.vision/?prefix=data/futures/um/daily/klines/BTCUSDT_220624/1h/
8 16 95
https://data.binance.vision/?prefix=data/futures/um/daily/klines/BTCUSDT_220930/1h/
9 16 78
https://data.binance.vision/?prefix=data/futures/um/daily/klines/ETHUSDT_210326/1h/
10 16 51
https://data.binance.vision/?prefix=data/futures/um/daily/klines/ETHUSDT_210625

In [4]:
_zip = dict(zip(futures_data_url_arr, zip_data_name_arr))

NameError: name 'futures_data_url_arr' is not defined

In [2]:
path = '/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/'
ls = os.listdir(path)



In [3]:
k = list(_zip.keys())[0]
v = _zip[k]
v

NameError: name '_zip' is not defined

In [21]:
colnames = ['openTime', 'open', 'high', 'low', 'close',
            'volume', 'closeTime', 'quoteAssetVolume', 'numberOfTrades', 'takerBuyBaseAssetVolume', 
            'takerBuyQuoteAssetVolume', 'ignore']

for i, k in enumerate(_zip.keys()):
    for v in _zip[k]:
        if (v.replace('.zip','.csv') in ls) == False:
            try:
                fileName = path+v.replace('.zip','.csv')
                print(fileName)
                url = (k+v).replace('?prefix=','')
                r = requests.get(url, stream=True)
                result = zipfile.ZipFile(BytesIO(r.content))
                df = pd.read_csv(result.open(result.namelist()[0]),
                                 names = colnames)     
                df.to_csv(fileName)
            except:
                continue
        
#         price.append(df)
#     print(len(price), len(_zip[k])) 
#     priceResults[v.split('-')[0]] = price
#     print(i,len(_zip.keys()), v.split('-')[0])

In [3]:
ls = [l for l in ls if l.endswith('.csv')]

In [ ]:
for fileName in ls[-10:]:
    print(fileName)
    fileName = path+fileName
    df = pd.read_csv(fileName, index_col=0)
    if df.iloc[0,0] == 'open_time':
        df = df.iloc[1:,:]
    df.openTime = pd.to_datetime(df.openTime*1000_000)
    df.closeTime = pd.to_datetime(df.closeTime*1000_000)


ETHUSDT_220930-1h-2022-08-27.csv


In [6]:
df.iloc[0,0] == 'open_time'

True

In [44]:
savePath = '/Volumes/external_SSD/copulaData/Binance_raw/futures_preprocessed/'

for fN in ls:
    print(fileName)
    fileName = path+fN
    df = pd.read_csv(fileName, index_col=0)
    if df.iloc[0,0] == 'open_time':
        df = df.iloc[1:,:]
    df.openTime = df.openTime.astype(int)*1000_000
    df.closeTime = df.closeTime.astype(int)*1000_000

    df.openTime = pd.to_datetime(df.openTime)
    df.closeTime = pd.to_datetime(df.closeTime)
    
    df.to_csv(savePath+fN)

/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCBUSD_210129-1h-2021-01-06.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCBUSD_210129-1h-2021-01-05.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCBUSD_210129-1h-2021-01-06.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCBUSD_210129-1h-2021-01-07.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCBUSD_210129-1h-2021-01-08.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCBUSD_210129-1h-2021-01-09.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCBUSD_210129-1h-2021-01-10.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCBUSD_210129-1h-2021-01-11.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCBUSD_210129-1h-2021-01-12.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCBUSD_210226-1h-2021-01-05.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCBUSD_210226-1h-2021-01-06.csv
/Volumes/e

/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_210625-1h-2021-04-13.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_210625-1h-2021-04-14.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_210625-1h-2021-04-15.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_210625-1h-2021-04-16.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_210625-1h-2021-04-17.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_210625-1h-2021-04-18.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_210625-1h-2021-04-19.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_210625-1h-2021-04-20.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_210625-1h-2021-04-21.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_210625-1h-2021-04-22.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_210625-1h-2021-04-23.csv
/Volumes/e

/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_210924-1h-2021-07-17.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_210924-1h-2021-07-18.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_210924-1h-2021-07-19.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_210924-1h-2021-07-20.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_210924-1h-2021-07-21.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_210924-1h-2021-07-22.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_210924-1h-2021-07-23.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_210924-1h-2021-07-24.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_210924-1h-2021-07-25.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_210924-1h-2021-07-26.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_210924-1h-2021-07-27.csv
/Volumes/e

/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_211231-1h-2021-10-18.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_211231-1h-2021-10-19.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_211231-1h-2021-10-20.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_211231-1h-2021-10-21.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_211231-1h-2021-10-22.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_211231-1h-2021-10-23.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_211231-1h-2021-10-24.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_211231-1h-2021-10-25.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_211231-1h-2021-10-26.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_211231-1h-2021-10-27.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_211231-1h-2021-10-28.csv
/Volumes/e

/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_220325-1h-2022-01-26.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_220325-1h-2022-01-27.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_220325-1h-2022-01-28.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_220325-1h-2022-01-29.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_220325-1h-2022-01-30.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_220325-1h-2022-01-31.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_220325-1h-2022-02-01.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_220325-1h-2022-02-02.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_220325-1h-2022-02-03.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_220325-1h-2022-02-04.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/BTCUSDT_220325-1h-2022-02-05.csv
/Volumes/e

...

/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_210924-1h-2021-07-19.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_210924-1h-2021-07-20.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_210924-1h-2021-07-21.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_210924-1h-2021-07-22.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_210924-1h-2021-07-23.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_210924-1h-2021-07-24.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_210924-1h-2021-07-25.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_210924-1h-2021-07-26.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_210924-1h-2021-07-27.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_210924-1h-2021-07-28.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_210924-1h-2021-07-29.csv
/Volumes/e

/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_211231-1h-2021-10-31.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_211231-1h-2021-11-01.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_211231-1h-2021-11-02.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_211231-1h-2021-11-03.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_211231-1h-2021-11-04.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_211231-1h-2021-11-05.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_211231-1h-2021-11-06.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_211231-1h-2021-11-07.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_211231-1h-2021-11-08.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_211231-1h-2021-11-09.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_211231-1h-2021-11-10.csv
/Volumes/e

/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220325-1h-2022-01-30.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220325-1h-2022-01-31.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220325-1h-2022-02-01.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220325-1h-2022-02-02.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220325-1h-2022-02-03.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220325-1h-2022-02-04.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220325-1h-2022-02-05.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220325-1h-2022-02-06.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220325-1h-2022-02-07.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220325-1h-2022-02-08.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220325-1h-2022-02-09.csv
/Volumes/e

/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220624-1h-2022-05-06.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220624-1h-2022-05-07.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220624-1h-2022-05-08.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220624-1h-2022-05-09.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220624-1h-2022-05-10.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220624-1h-2022-05-11.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220624-1h-2022-05-12.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220624-1h-2022-05-13.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220624-1h-2022-05-14.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220624-1h-2022-05-15.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220624-1h-2022-05-16.csv
/Volumes/e

/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220930-1h-2022-08-22.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220930-1h-2022-08-23.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220930-1h-2022-08-24.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220930-1h-2022-08-25.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220930-1h-2022-08-26.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220930-1h-2022-08-27.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220930-1h-2022-08-28.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220930-1h-2022-08-29.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220930-1h-2022-08-30.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220930-1h-2022-08-31.csv
/Volumes/external_SSD/copulaData/Binance_raw/futures_raw/ETHUSDT_220930-1h-2022-09-01.csv
/Volumes/e

In [45]:
df

,openTime,open,high,low,close,volume,closeTime,quoteAssetVolume,numberOfTrades,takerBuyBaseAssetVolume,takerBuyQuoteAssetVolume,ignore
1,2022-09-05 00:00:00,1558.80,1573.60,1549.95,1554.87,2051.120,2022-09-05 00:59:59.999,3200501.64051,2001,920.001,1435778.55077,0
2,2022-09-05 01:00:00,1554.61,1562.96,1549.56,1552.27,1132.215,2022-09-05 01:59:59.999,1762455.39235,1105,501.873,781333.17900,0
3,2022-09-05 02:00:00,1552.27,1557.54,1547.68,1556.10,641.848,2022-09-05 02:59:59.999,997290.72437,658,402.643,625793.23524,0
4,2022-09-05 03:00:00,1556.47,1558.00,1549.30,1554.56,846.443,2022-09-05 03:59:59.999,1316096.42728,771,294.403,457668.56804,0
5,2022-09-05 04:00:00,1555.12,1559.90,1552.20,1556.76,914.176,2022-09-05 04:59:59.999,1422741.09917,792,480.836,748271.74904,0
6,2022-09-05 05:00:00,1557.08,1557.46,1535.39,1541.03,2254.398,2022-09-05 05:59:59.999,3481024.23106,1695,982.752,1518385.09329,0
7,2022-09-05 06:00:00,1540.45,1549.53,1537.57,1546.84,712.761,2022-09-05 06:59:59.999,1100608.32530,956,414.341,639678.89830,0
8,2022-09-05 07:00:00,1547.18,1547.32,1536.45,1541.78,1262.594,2022-09-05 07:59:59.999,1945920.85135,869,606.099,933547.03861,0
9,2022-09-05 08:00:00,1541.78,1546.76,1537.81,1545.02,1273.049,2022-09-05 08:59:59.999,1964315.39479,989,769.456,1187404.29308,0
10,2022-09-05 09:00:00,1544.86,1548.64,1539.84,1544.69,994.313,2022-09-05 09:59:59.999,1535708.54765,811,430.000,664207.74109,0


In [38]:
pd.to_datetime(df.openTime)

1    2022-09-04 00:00:00
2    2022-09-04 01:00:00
3    2022-09-04 02:00:00
4    2022-09-04 03:00:00
5    2022-09-04 04:00:00
6    2022-09-04 05:00:00
7    2022-09-04 06:00:00
8    2022-09-04 07:00:00
9    2022-09-04 08:00:00
10   2022-09-04 09:00:00
11   2022-09-04 10:00:00
12   2022-09-04 11:00:00
13   2022-09-04 12:00:00
14   2022-09-04 13:00:00
15   2022-09-04 14:00:00
16   2022-09-04 15:00:00
17   2022-09-04 16:00:00
18   2022-09-04 17:00:00
19   2022-09-04 18:00:00
20   2022-09-04 19:00:00
21   2022-09-04 20:00:00
22   2022-09-04 21:00:00
23   2022-09-04 22:00:00
24   2022-09-04 23:00:00
Name: openTime, dtype: datetime64[ns]

In [ ]:
priceResults_cleaned = dict()
for name in priceResults:
    priceResults_cleaned[name]=pd.concat(priceResults[name], axis=0, ignore_index=True)

In [ ]:
externalDisk = '/Volumes/external_SSD/copulaData/'
for name in priceResults:
    priceResults_cleaned[name].to_pickle(externalDisk+'Binance/'+'binance'+name+'.pickle')    

In [ ]:
print(name)
pd.read_pickle(externalDisk+'Binance/'+'binance'+name+'.pickle')

## Spot data

In [46]:
spotNames = ['ADAUSDT', 'BTCUSDT', 'ETHUSDT', 'LTCUSDT', 'XRPUSDT']

In [75]:
url = 'https://data.binance.vision/?prefix=data/futures/um/monthly/klines/'


spot_data_url_arr = []
for spotName in spotNames:
    spot_data_url_arr.append(url+spotName+'/1h/')

In [87]:
zip_data_name_arr = []
driver = webdriver.Chrome(ChromeDriverManager().install())

i=0
length = 0
for data_url in spot_data_url_arr:
    print(data_url)
    driver.get(data_url)
    driver.implicitly_wait(5)
    while length == 0 :
        print(length)
        _list = driver.find_element_by_id('listing')
        rows = _list.find_elements(By.TAG_NAME, 'tr')
        names = [row.find_elements(By.TAG_NAME, 'td')[0].text for row in rows]    
        names = [name for name in names if name.endswith('.zip')]
        zip_data_name_arr.append(names)
        length = len(names)
    length = 0
    i+=1
    print(i, len(spot_data_url_arr), len(names))

https://data.binance.vision/?prefix=data/futures/um/monthly/klines/ADAUSDT/1h/
0
1 5 32
https://data.binance.vision/?prefix=data/futures/um/monthly/klines/BTCUSDT/1h/
0
2 5 32
https://data.binance.vision/?prefix=data/futures/um/monthly/klines/ETHUSDT/1h/
0
3 5 32
https://data.binance.vision/?prefix=data/futures/um/monthly/klines/LTCUSDT/1h/
0
4 5 32
https://data.binance.vision/?prefix=data/futures/um/monthly/klines/XRPUSDT/1h/
0
5 5 32


In [88]:
_zip = dict(zip(spot_data_url_arr, zip_data_name_arr))

In [125]:
path = '/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/'
ls = os.listdir(path)

In [113]:
colnames = ['openTime', 'open', 'high', 'low', 'close',
            'volume', 'closeTime', 'quoteAssetVolume', 'numberOfTrades', 'takerBuyBaseAssetVolume', 
            'takerBuyQuoteAssetVolume', 'ignore']

for i, k in enumerate(_zip.keys()):
    for v in _zip[k]:
        if (v.replace('.zip','.csv') in ls)== False:
            try:
                fileName = path+v.replace('.zip','.csv')
                print(fileName)
                url = (k+v).replace('?prefix=','')
                r = requests.get(url, stream=True)
                result = zipfile.ZipFile(BytesIO(r.content))
                df = pd.read_csv(result.open(result.namelist()[0]),
                                 names = colnames)     
                df.to_csv(fileName)
            except:
                continue

/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/ADAUSDT-1h-2022-08.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/ADAUSDT-1h-2022-07.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/ADAUSDT-1h-2022-06.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/ADAUSDT-1h-2022-05.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/ADAUSDT-1h-2022-04.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/ADAUSDT-1h-2022-03.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/ADAUSDT-1h-2022-02.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/ADAUSDT-1h-2022-01.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/ADAUSDT-1h-2021-12.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/ADAUSDT-1h-2021-11.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/ADAUSDT-1h-2021-10.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/ADAUSDT-1h-2021-09.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/ADAUSDT-1h-2021-08.csv

/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/LTCUSDT-1h-2021-09.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/LTCUSDT-1h-2021-08.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/LTCUSDT-1h-2021-07.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/LTCUSDT-1h-2021-06.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/LTCUSDT-1h-2021-05.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/LTCUSDT-1h-2021-04.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/LTCUSDT-1h-2021-03.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/LTCUSDT-1h-2021-02.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/LTCUSDT-1h-2021-01.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/LTCUSDT-1h-2020-12.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/LTCUSDT-1h-2020-11.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/LTCUSDT-1h-2020-10.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/LTCUSDT-1h-2020-09.csv

In [127]:
savePath = '/Volumes/external_SSD/copulaData/Binance_raw/spot_preprocessed/'

for fN in ls:
    print(fileName)
    fileName = path+fN
    df = pd.read_csv(fileName, index_col=0)
    if df.iloc[0,0] == 'open_time':
        df = df.iloc[1:,:]
    df.openTime = df.openTime.astype(int)*1000_000
    df.closeTime = df.closeTime.astype(int)*1000_000

    df.openTime = pd.to_datetime(df.openTime)
    df.closeTime = pd.to_datetime(df.closeTime)
    
    df.to_csv(savePath+fN)

/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/XRPUSDT-1h-2020-01.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/ADAUSDT-1h-2020-01.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/ADAUSDT-1h-2020-02.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/ADAUSDT-1h-2020-03.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/ADAUSDT-1h-2020-04.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/ADAUSDT-1h-2020-05.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/ADAUSDT-1h-2020-06.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/ADAUSDT-1h-2020-07.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/ADAUSDT-1h-2020-08.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/ADAUSDT-1h-2020-09.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/ADAUSDT-1h-2020-10.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/ADAUSDT-1h-2020-11.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/ADAUSDT-1h-2020-12.csv

/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/LTCUSDT-1h-2021-02.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/LTCUSDT-1h-2021-03.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/LTCUSDT-1h-2021-04.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/LTCUSDT-1h-2021-05.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/LTCUSDT-1h-2021-06.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/LTCUSDT-1h-2021-07.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/LTCUSDT-1h-2021-08.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/LTCUSDT-1h-2021-09.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/LTCUSDT-1h-2021-10.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/LTCUSDT-1h-2021-11.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/LTCUSDT-1h-2021-12.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/LTCUSDT-1h-2022-01.csv
/Volumes/external_SSD/copulaData/Binance_raw/spot_raw/LTCUSDT-1h-2022-02.csv

## Combine

In [197]:
columns = ['openTime', 'open', 'high', 'low', 'close', 'instrumentName', 'quoteAssetVolume']

In [256]:
spotPath = '/Volumes/external_SSD/copulaData/Binance_raw/spot_preprocessed/'
futuresPath = '/Volumes/external_SSD/copulaData/Binance_raw/futures_preprocessed/'

spotpair = 'BTCUSDT'
futurespair = 'BTCUSDT'
for spotpair in spotNames:
    for futurespair in ['BTCUSDT', 'ETHUSDT']:
        print(spotpair, futurespair, 'start')
        spot_arr = [l for l in os.listdir(spotPath) if l.endswith('.csv') & l.startswith(spotpair)]
        futures_arr = [l for l in os.listdir(futuresPath) if l.endswith('.csv') & l.startswith(futurespair)]
        print('number of spot price files to process:', 
              len(spot_arr),
              '\nnumber of futures price files to process:', 
              len(futures_arr))
        results = []

        for spotFile in spot_arr:
            spotName = spotFile[:spotFile.find('-')]
            filePath = spotPath+spotFile
            spot = pd.read_csv(filePath, index_col=0)
            spot.loc[:, 'instrumentName'] = spotName
            spot = spot.loc[:, columns]
            spot.loc[:, 'instrumentType'] = 'spot'
            results.append(spot)

        for futuresFile in futures_arr:
            futuresName = futuresFile[:futuresFile.find('-')]
            filePath = futuresPath+futuresFile
            futures = pd.read_csv(filePath, index_col=0)
            futures.loc[:, 'instrumentName'] = futuresName
            futures = futures.loc[:, columns]
            futures.loc[:, 'instrumentType'] = 'futures'
            results.append(futures)

        results = pd.concat(results, axis=0)
        results.reset_index(drop=True, inplace=True)
        results.openTime = pd.to_datetime(results.openTime)
        time_arr = sorted(list(set(results.openTime)))

        cleanData = dict()
        for i, t in enumerate(time_arr):
            if i%500==0:
                print(i, len(time_arr))
            _id = results.openTime == t

            _data = results.loc[_id, :]

            s_id = _data.instrumentType == 'spot'
            if sum(s_id) != 0:
                row = list(_data.loc[s_id, 'close'])
            else:
                row = [None]

            f_id = _data.instrumentType == 'futures'
            if sum(f_id) != 0:
                futuresClose = _data.loc[f_id, ['close', 'quoteAssetVolume', 'instrumentName']]
                futuresClose.sort_values('quoteAssetVolume', inplace=True)
                row += list(futuresClose.iloc[-1, [0,2]])
            else:
                row += [None, None]
            cleanData[t] = row

        # clean
        cleanData_df = pd.DataFrame(cleanData).T
        cleanData_df.columns = ['spot', 'futures', 'futuresName']
        _id = cleanData_df.isna().sum(axis=1) == 0
        cleanData_df = cleanData_df.loc[_id, :]
        print('cleaned')

        # save
        savePath = '/Volumes/external_SSD/copulaData/Binance_cleaned/'
        cleanData_df.to_csv(savePath + spotpair+'_'+futurespair+'.csv')
        print('saved!')

ADAUSDT BTCUSDT start
number of spot price files to process: 32 
number of futures price files to process: 618
0 22768
500 22768
1000 22768
1500 22768
2000 22768
2500 22768
3000 22768
3500 22768
4000 22768
4500 22768
5000 22768
5500 22768
6000 22768
6500 22768
7000 22768
7500 22768
8000 22768
8500 22768
9000 22768
9500 22768
10000 22768
10500 22768
11000 22768
11500 22768
12000 22768
12500 22768
13000 22768
13500 22768
14000 22768
14500 22768
15000 22768
15500 22768
16000 22768
16500 22768
17000 22768
17500 22768
18000 22768
18500 22768
19000 22768
19500 22768
20000 22768
20500 22768
21000 22768
21500 22768
22000 22768
22500 22768
cleaned
saved!
ADAUSDT ETHUSDT start
number of spot price files to process: 32 
number of futures price files to process: 617
0 22768
500 22768
1000 22768
1500 22768
2000 22768
2500 22768
3000 22768
3500 22768
4000 22768
4500 22768
5000 22768
5500 22768
6000 22768
6500 22768
7000 22768
7500 22768
8000 22768
8500 22768
9000 22768
9500 22768
10000 22768
10500 2

In [250]:
cleanData_df.to_csv(savePath + spotpair+'_'+futurespair)

In [253]:
cleanData_df = pd.DataFrame(cleanData).T

In [254]:
cleanData_df

,0,1,2
2020-01-31 08:00:00,0.05458,NaN,NaN
2020-01-31 09:00:00,0.05473,NaN,NaN
2020-01-31 10:00:00,0.05432,NaN,NaN
2020-01-31 11:00:00,0.05414,NaN,NaN
2020-01-31 12:00:00,0.05252,NaN,NaN
...,...,...,...
2021-12-07 13:00:00,1.43700,NaN,NaN
2021-12-07 14:00:00,1.45070,NaN,NaN
2021-12-07 15:00:00,1.43180,NaN,NaN
2021-12-07 16:00:00,1.41450,NaN,NaN
